In [ ]:
%config Completer.use_jedi = False

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### SQL with pandas (create database)

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
agency = pd.read_csv('sem_gtfs/agency.txt')
stoptimes = pd.read_csv('sem_gtfs/stop_times.txt')
stops = pd.read_csv('sem_gtfs/stops.txt')
trips = pd.read_csv('sem_gtfs/trips.txt')

##### Add one row

In [ ]:
def gen_insert_query(table_name:str, a_dict:dict) -> str:
    return f"INSERT INTO {table_name} {tuple(a_dict.keys())} VALUES {tuple(a_dict.values())}"

##### Add rows from dict

In [ ]:
def gen_insert_queries(table_name:str, a_dict:dict) -> str:
    return f"INSERT INTO {table_name} {tuple(a_dict.keys())} VALUES {','.join(str(i) for i in tuple(zip(*a_dict.values())))}"

##### Add rows from pandas DataFrame

In [ ]:
def gen_insert_queries_df(table_name:str, df:pd.DataFrame) -> str:
    dico = df.to_dict(orient='list')
    return f"INSERT INTO {table_name} {tuple(dico.keys())} VALUES {','.join(str(i) for i in tuple(zip(*dico.values())))}"

In [ ]:
dico = dict(trips.iloc[0])
dico_multi = trips.loc[:, ['trip_id', 'trip_headsign', 'service_id', 'direction_id']].iloc[20:30].to_dict(orient='list')

In [ ]:
gen_insert_query('trips', dico)

In [ ]:
gen_insert_queries_df('trips', trips.iloc[:10])

##### Execute SQL insertion commands

In [ ]:
conn = sqlite3.connect('gtfs_tag.db')
cur = conn.cursor()
cur.execute(gen_insert_queries('trips', dico_multi))
conn.commit()
conn.close()

#####  Create file containing sql insertion commands

In [ ]:
def gen_insert_file(filename, tablename, df):
    with open(filename, 'w') as file:
        file.write(gen_insert_queries_df(tablename, df))

In [ ]:
gen_insert_file('file_test.txt', 'trips', trips.loc[:, ['trip_id', 'trip_headsign', 'service_id', 'direction_id']].iloc[:10])

##### Execute from file

In [ ]:
def execute_sql_file(db_name, file_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    with open(file_name) as file:
        cur.execute(file.read())
    conn.commit()
    conn.close()